In [2]:
MODELS_PATH = "/root/.cache/huggingface/zzl/models/cosyvoice"

In [3]:
import sys
sys.path.append('third_party/Matcha-TTS')
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio

2025-02-25 10:26:14,539 - modelscope - INFO - PyTorch version 2.3.1+cu121 Found.
2025-02-25 10:26:14,542 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-02-25 10:26:14,665 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 2a832d566ee6d1ad18205c50f7781c34 and a total number of 980 components indexed
/opt/conda/envs/cosyvoice/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CosyVoice2 Usage 

In [4]:
# import logging

# logging.basicConfig(level=logging.ERROR)

try:
    cosyvoice = CosyVoice2(f"{MODELS_PATH}/pretrained_models/CosyVoice2-0.5B", load_jit=False, load_trt=False, fp16=False)
except Exception as e:
    import traceback
    print("ERROR: ", traceback.format_exc())    

/opt/conda/envs/cosyvoice/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-25 10:26:43,035 INFO input frame rate=25
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-02-25 10:26:50.426492161 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 8 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-02-25 10:26:50.429225403 [W

text.cc: festival_Text_init
open voice lang map failed


In [5]:
# zero_shot usage
prompt_speech_16k = load_wav('./asset/zero_shot_prompt.wav', 16000)
for i, j in enumerate(cosyvoice.inference_zero_shot(
    '收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', 
    # " vector quantization（向量量化）是一种将连续的高维数据映射到一个离散的低维空间的技术。具体来说，它将输入的连续数据（比如语音信号或特征向量）映射到一个预先定义的离散“字典”中，这些字典中的元素通常是有限的，因此这能减少输入数据的复杂度，并对数据进行压缩或离散化。",
    # "在RAG系统中，这种高效的实现往往是通过“分块”来实现的。你可以把它想象成把一本厚书分成几章——这样一来，阅读和理解就轻松多了。同样地，分块技术把大段复杂的文本拆分成更小、更容易处理的片段，让AI能更快、更准确地理解和处理信息。",
    '希望你以后能够做的比我还好呦。',
    prompt_speech_16k, 
    stream=False,
    )):
    torchaudio.save('./generate_speech/zero_shot_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:28:15,043 INFO synthesis text 收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。
2025-02-25 10:28:24,411 INFO yield speech len 12.32, rtf 0.7603888000760759
100%|██████████| 1/1 [00:13<00:00, 13.05s/it]


In [6]:
# fine grained control
for i, j in enumerate(cosyvoice.inference_cross_lingual(
    '在他讲述那个荒诞故事的过程中，他突然[laughter]停下来，因为他自己也被逗笑了[laughter]。',
    prompt_speech_16k,
    stream=False,
)):
    torchaudio.save('./generate_speech/fine_grained_control_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:28:28,479 INFO synthesis text 在他讲述那个荒诞故事的过程中，他突然[laughter]停下来，因为他自己也被逗笑了[laughter]。
2025-02-25 10:28:31,938 INFO yield speech len 7.8, rtf 0.4434241087008745
100%|██████████| 1/1 [00:03<00:00,  3.57s/it]


In [7]:
# instruct usage
for i, j in enumerate(cosyvoice.inference_instruct2(
    '收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。',
    '用粤语说这句话',
    prompt_speech_16k,
    stream=False,
)):
    torchaudio.save('./generate_speech/instruct_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:28:35,710 INFO synthesis text 收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。
2025-02-25 10:28:40,339 INFO yield speech len 10.6, rtf 0.43673146445796174
100%|██████████| 1/1 [00:04<00:00,  4.70s/it]


In [8]:
# bistream usage
# 使用双流时，您可以使用生成器作为输入，这在使用文本 llm 模型作为输入时非常有用
# 注意：由于 llm 无法处理任意长度的句子，因此您仍需掌握一些基本的句子分割逻辑
def text_generator():
    yield '收到好友从远方寄来的生日礼物，'
    yield '那份意外的惊喜与深深的祝福'
    yield '让我心中充满了甜蜜的快乐，'
    yield '笑容如花儿般绽放。'

for i, j in enumerate(cosyvoice.inference_zero_shot(
    text_generator(),
    '希望你以后能够做的比我还好呦。',
    prompt_speech_16k,
    stream=False,
)):    
    torchaudio.save('./generate_speech/bistream_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

2025-02-25 10:28:41,792 INFO get tts_text generator, will skip text_normalize!
  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:28:41,796 INFO get tts_text generator, will return _extract_text_token_generator!
2025-02-25 10:28:41,868 INFO synthesis text <generator object text_generator at 0x716dbafc6d50>
2025-02-25 10:28:41,872 INFO append 5 text token 15 speech token
2025-02-25 10:28:41,873 INFO append 5 text token 15 speech token
2025-02-25 10:28:41,874 INFO append 5 text token 15 speech token
2025-02-25 10:28:41,875 INFO not enough text token to decode, wait for more
2025-02-25 10:28:41,876 INFO not enough text token to decode, wait for more
2025-02-25 10:28:41,878 INFO not enough text token to decode, wait for more
2025-02-25 10:28:41,879 INFO not enough text token to decode, wait for more
2025-02-25 10:28:41,880 INFO append 5 text token 15 speech token
2025-02-25 10:28:41,881 INFO not enough text token to decode, wait for more
2025-02-25 10:28:41,882 INFO not enough text token t

## CosyVoice Usage

In [9]:
cosyvoice1 = CosyVoice(
    f"{MODELS_PATH}/pretrained_models/CosyVoice-300M-SFT",
    load_jit=False,
    load_trt=False, 
    fp16=False,
    )

2025-02-25 10:28:52,481 INFO input frame rate=50
2025-02-25 10:28:58.060171249 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 12 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-02-25 10:28:58.061505436 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-02-25 10:28:58.061514878 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


open voice lang map failed


festival_initialize() called more than once


In [10]:
# sft usage
print(cosyvoice1.list_available_spks())

['中文女', '中文男', '日语男', '粤语女', '英文女', '英文男', '韩语女']


In [11]:
# change stream=True for chunk stream inference
for i,j in enumerate(cosyvoice1.inference_sft(
    '你好，我是通义生成式语音大模型，请问有什么可以帮您的吗？',
    '中文女',
    stream=False,
)):
    torchaudio.save('./generate_speech/cosyvoice1/sft_{}.wav'.format(i), j['tts_speech'], cosyvoice1.sample_rate)

  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:29:24,294 INFO synthesis text 你好，我是通义生成式语音大模型，请问有什么可以帮您的吗？
2025-02-25 10:29:27,380 INFO yield speech len 5.0967800453514736, rtf 0.6054171600754549
100%|██████████| 1/1 [00:03<00:00,  3.10s/it]


In [17]:
cosyvoice1 = CosyVoice(f'{MODELS_PATH}/pretrained_models/CosyVoice-300M')
# zero-shot usage
prompt_speech_16k = load_wav('./asset/zero_shot_prompt.wav', 16000)
for i, j in enumerate(cosyvoice1.inference_zero_shot(
    '收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。',
    '希望你以后能够做的比我还好呦。',
    prompt_speech_16k,
    stream=False,
)):
    torchaudio.save('./generate_speech/cosyvoice1/zero_shot_{}.wav'.format(i), j['tts_speech'], cosyvoice1.sample_rate)

2025-02-25 10:39:13,092 INFO input frame rate=50
2025-02-25 10:39:14.097100357 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 12 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-02-25 10:39:14.098439293 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-02-25 10:39:14.098448068 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
festival_initialize() called more than once


open voice lang map failed


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: stft(torch.FloatTensor[1, 77566], n_fft=1024, hop_length=256, win_length=1024, window=torch.FloatTensor{[1920]}, normalized=0, onesided=1, return_complex=1) : expected a 1D window tensor of size equal to win_length=1024, but got window with size [1920]

In [21]:
# cross-lingual usage
prompt_speech_16k = load_wav('./asset/cross_lingual_prompt.wav', 16_000)
for i, j in enumerate(cosyvoice1.inference_cross_lingual(
    '<|en|>And then later on, fully acquiring that company. So keeping management in line, interest in line with the asset that\'s coming into the family is a reason why sometimes we don\'t buy the whole thing.',
    prompt_speech_16k,
    stream=False,
)):
    torchaudio.save('./generate_speech/cosyvoice1/cross_linugal_{}.wav'.format(i), j['tts_speech'], sample_rate=cosyvoice1.sample_rate)

  0%|          | 0/1 [00:00<?, ?it/s]

min value is  tensor(-1.0054)


RuntimeError: stft(torch.FloatTensor[1, 303949], n_fft=1024, hop_length=256, win_length=1024, window=torch.FloatTensor{[1920]}, normalized=0, onesided=1, return_complex=1) : expected a 1D window tensor of size equal to win_length=1024, but got window with size [1920]

In [19]:
# vc usage
prompt_speech_16k = load_wav('./asset/zero_shot_prompt.wav', 16000)
source_speech_16k = load_wav('./asset/cross_lingual_prompt.wav', 16000)
for i, j in enumerate(cosyvoice1.inference_vc(
    source_speech_16k, 
    prompt_speech_16k,
    stream=False,
    )):
    torchaudio.save("./generate_speech/cosyvoice1/vc_{}".format(i), j['tts_speech'], sample_rate=cosyvoice1.sample_rate)

RuntimeError: stft(torch.FloatTensor[1, 77566], n_fft=1024, hop_length=256, win_length=1024, window=torch.FloatTensor{[1920]}, normalized=0, onesided=1, return_complex=1) : expected a 1D window tensor of size equal to win_length=1024, but got window with size [1920]

In [22]:
cosyvoice1 = CosyVoice('{}/pretrained_models/CosyVoice-300M-Instruct'.format(MODELS_PATH), load_jit=False, load_trt=False, fp16=False)
# instruct usage
for i, j in enumerate(cosyvoice1.inference_instruct(
    '在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。',
    '中文女',
    'Theo \'Crimson\', is a fiery, passionate rebel leader. Fights with fervor for justice, but struggles with impulsiveness.',
    stream=False,
)):
    torchaudio.save('./generate_speech/cosyvoice1/instruct_{}.wav'.format(i), j['tts_speech'], cosyvoice1.sample_rate)

2025-02-25 10:55:31,053 INFO input frame rate=50
2025-02-25 10:55:31.795762984 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 12 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-02-25 10:55:31.797126065 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-02-25 10:55:31.797135432 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
festival_initialize() called more than once


open voice lang map failed


  0%|          | 0/1 [00:00<?, ?it/s]2025-02-25 10:55:33,938 INFO synthesis text 在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。
2025-02-25 10:55:36,906 INFO yield speech len 5.491519274376417, rtf 0.540442898255196
100%|██████████| 1/1 [00:03<00:00,  3.07s/it]
